# Multi-Lender Symmetric Funnel Decomposition Analysis

This notebook demonstrates **multi-lender symmetric decomposition** showing:
- Aggregate waterfall charts with lender-level breakdowns
- Side-by-side comparison: Overall vs By Lender
- Drilldown charts showing each effect split by lender

## Workflow
1. Load booking funnel data  
2. Calculate decomposition for each lender
3. Visualize with 2-panel waterfall grid
4. Compare lender contributions

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import symmetric_decomposition_calculator
import visualization_engine

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 8)
plt.rcParams['figure.dpi'] = 100

## 1. Load Data

Using the standard mock_v2 dataset with ACA lender:

In [ ]:
# Load data
data_path = Path.cwd().parent / 'data' / 'funnel_data_mock_v2.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

print(f"Loaded {len(df)} rows of data")
print(f"\nLenders: {', '.join(sorted(df['lender'].unique()))}")
print(f"Date range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Unique months: {df['month_begin_date'].nunique()}")

## 2. Calculate Multi-Lender Symmetric Decomposition

Comparing **June 2023 → June 2024** (Year-over-Year):

In [ ]:
# Calculate multi-lender decomposition
results = symmetric_decomposition_calculator.calculate_multi_lender_decomposition(
    df=df,
    date_a='2023-06-01',
    date_b='2024-06-01'
)

print("Multi-Lender Symmetric Decomposition Results:")
print(f"  Method: {results.metadata['method']}")
print(f"  Period 1: {results.metadata['date_a']}")
print(f"  Period 2: {results.metadata['date_b']}")
print(f"  Lenders analyzed: {', '.join(results.metadata['lenders'])}")
print(f"\nAggregate Results (All Lenders):")
print(f"  Period 1 bookings: {results.metadata['aggregate_period_1_bookings']:,.0f}")
print(f"  Period 2 bookings: {results.metadata['aggregate_period_2_bookings']:,.0f}")
print(f"  Delta bookings: {results.metadata['aggregate_delta_bookings']:+,.0f}")

## 3. View Aggregate Summary

In [ ]:
results.aggregate_summary

## 4. Create Lender Waterfall Grid

**2-Panel Visualization:**
- **Left Panel**: Overall aggregate waterfall (no lender distinction)
- **Right Panel**: Lender breakdown waterfall (stacked bars by lender)

This allows direct comparison between the overall view and the lender-attributed view.

In [ ]:
# Create lender waterfall grid
fig = visualization_engine.create_lender_waterfall_grid(
    lender_summaries=results.lender_summaries,
    aggregate_summary=results.aggregate_summary,
    metadata=results.metadata
)

plt.show()

## 5. Print Lender Breakdowns (Tabular View)

In [ ]:
visualization_engine.print_lender_breakdowns(results.lender_summaries)

## 6. Create Lender Drilldown Charts

Shows **each effect broken down by lender** in horizontal bar format:

In [ ]:
# Create lender drilldown
fig = visualization_engine.create_lender_drilldown(
    lender_summaries=results.lender_summaries,
    date_a=results.metadata['date_a'],
    date_b=results.metadata['date_b']
)

plt.show()

## Summary

This multi-lender analysis provides:

### Key Features:
1. **2-Panel Grid**: Compare overall vs lender-attributed views side-by-side
2. **Lender Attribution**: See which lender drives each effect
3. **Consistent Formatting**: Matches the style of single-lender waterfalls
4. **Flexible**: Works with any number of lenders in the dataset

### Use Cases:
- **Portfolio Management**: Understand overall lender network performance
- **Lender Comparison**: Identify top and bottom performers
- **Strategic Planning**: Allocate resources based on lender contributions
- **Root Cause Analysis**: Determine if issues are lender-specific or systematic